# Dr. Hamm's Checkers Game


## Getting Started
Checkers is typically played on the same 8x8 board as chess. In order to play the game, we are going to have to store the state of the game, i.e. the contents of each square, in memory. A natural representation of the board is a matrix, where every element corresponds to a square on the board. We choose a convention that the square is empty when corresponding matrix element is 0, and 1 or 2 when there is piece from player 1 or 2, respectively, in the box.

We may choose to change this convention later, so instead of "hard-wiring" 0, 1, and 2 into our code, we'll use variables that store the index. Using a variable will also make our code more readable. Similarily, we will keep our size flexible.

In [1]:
# Assignment values for the board representation
player_1 = 1
player_2 = 2
empty = 0

# Specify game board size
size = 8

## Make a starting board
Now lets write a function that will create a new game board for us with the pieces in the right spot. There are various ways to put the pieces into place. One observation is that only six rows are filled, and that each player starts with their piece in the bottom left corner of the board (from their perspective).  In Lab, we realized that to have a fair game, the board size must be an even integer greater than 6 (6 or less and we can't give each player 3 rows, and for odd sizes, one player will have more pieces than the other).

We add Player 1's pieces at the bottom of the board and Player 2's at the top of the board. Note that we cound from the bottom to add Player 1's pieces and from the top to add Player 2's. For each row, we increment by 2 until the end of the row, adding a piece at every other location.

In [2]:
def make_game_board(size=8):
    # Check if the board size is an even integer >= 8. If not return an empty board and an error
    if isinstance(size,int)==False or (size%2)!=0 or size<8:
        print('Error: board size must be an even integer greater than or equal to 8')
        return list()
    
    # Make an empty board
    board = [[empty]*size for i in range(size)]
    
    # Add Player pieces
    for i in range(0,size,2):
        # Player 1 pieces
        board[-1][i] = player_1
        board[-2][i+1] = player_1
        board[-3][i] = player_1
        # Player 2 pieces
        board[0][i+1] = player_2
        board[1][i] = player_2
        board[2][i+1] = player_2
    
    return board

# Write a function to handle player moves
We note that checkers is a simple game where each (non-king) piece moves forward and to the left or right. So we can specify a move by choosing a piece and picking left or right. A move is simply taking a piece at location `x,y` to a location `x+x_offset,y+y_offset`, where the offsets depend on the player and the choosen direction. So the first step in making a moving a piece is to determine what is the appropriate offset for given the color (aka player number) and the direction of intended move (right or left).

This `moves` dictionary is an elegant and flexible solution to do this. We can change the definitions of `player_1`, `player_2`, `right_move`, and `left_move` without having to change the definition of `moves`. The message to walk away from this small discussion is that there sometimes are various ways to represent logic. In this case as a function, list of lists, or dictionary of dictionaries. And some provide more flexibility and/or are more elegant than others.

In [3]:
left_move=0
right_move=1

player_1_left_move = (-1,-1)
player_1_right_move = (-1,1)
player_2_left_move = (1,1)
player_2_right_move = (1,-1)

## Create a dictionary of dictionaries
moves = {player_1: {left_move: player_1_left_move,
                  right_move: player_1_right_move},
        player_2: {left_move: player_2_left_move,
                  right_move: player_2_right_move}}
moves

{1: {0: (-1, -1), 1: (-1, 1)}, 2: {0: (1, 1), 1: (1, -1)}}

### Moving Pieces

Next let's code up a function that will take a board, player, location of a piece, and desired move and then returns a new board with the desired moved made. This function is going to be the heart of the game in a sense. It captures the rules of what players can and cannot do.

Quick aside: We are going to put in some print statements so we see what is happening, but we don't necessarily want to see these statements all of the time, for example later we may wish to write an artificial intelligence that uses the same function to think about the game, and we  don't necessarily see everything it's thinking. So lets write a function that we can use for printing

In [4]:
def print_message(message, verbose=True):
    if verbose:
        print(message)

In [5]:
print_message("hello world")

hello world


In [6]:
print_message("hello world",False)

Logical progression of this function:
1. Check if the player has a piece at the given location
1. Look up the offset for a valid move (from dictionary)
1. Check if the move (and possible jump move) is on the board
1. If the target space is on the board and empty, move the player's piece by emptying the starting position and placing the piece into the target space
1. If the target space is filled with the opponent's piece, but the next space along the diagonal is empty, then empty the start space, remove the target, and place the piece in the next space along the diagonal

We will return `True` if the move is valid, and `False` if the move is invalid

In [7]:
def move_piece(board,player,location,move,verbose=True):
    x,y = location
    
    # Check if player has a piece at location
    if not board[x][y] == player:
        print_message("Player does not have a piece at this location",verbose)
        return False
    
    # Look up the offset for a valid move
    x_offset,y_offset = moves[player][move]
    
    # Check that the move is on the board
    move_possible = x + x_offset < size and \
                    x + x_offset >= 0 and \
                    y + y_offset < size and \
                    y + y_offset >= 0
    #print_message("move_possible is {}".format(move_possible),verbose)
    
    jump_possible = x + 2*x_offset < size and \
                    x + 2*x_offset >= 0 and \
                    y + 2*y_offset < size and \
                    y + 2*y_offset >= 0
    #print_message("jump_possible is {}".format(jump_possible),verbose)
    
    if not (move_possible or jump_possible):
        print_message("Move is off of the board.",verbose)
        return False
    
    # Try to make the move
    # Check if the target space is empty
    if move_possible and board[x+x_offset][y+y_offset] == empty:
        # Make the move
        # Empty the current spot
        board[x][y] = empty
        # Add player piece in the target space
        board[x+x_offset][y+y_offset] = player
        print_message("Player's piece has been moved.",verbose)
        
        return True
    
    # Does the target space have an opponent's piece, and the space after is empty
    elif jump_possible and board[x+x_offset][y+y_offset]!=player and board[x+2*x_offset][y+2*y_offset]==empty:
        # Make the move
        # Empty the current spot
        board[x][y] = empty
        # Remove the opponent's piece
        board[x+x_offset][y+y_offset] = empty
        # Move player to the new spot (2 away along the diagonal)
        board[x+2*x_offset][y+2*y_offset] = player
        print_message("Player took opponent's piece.",verbose)
        
        return True
    
    else:
        print_message("Move is invalid.",verbose)
        return False    

## Making a prettier gameboard display
It's hard to "see" the game board using a printout of the list of lists. Lets "draw" one instead using other characters. 

First lets define what characters represent each type of space:

In [8]:
player_1_piece = "X"
player_2_piece = "O"
empty_space = " "

In [ ]:
# What piece do I put in a given board space?
space_character = {player_1: player_1_piece,
                  player_2: player_2_piece,
                  empty: empty_space}
space_character

{1: 'X', 2: 'O', 0: ' '}

Now we can print our board by looping over its matrix representation and using `print` to write the correct characters in the right spot.

Note the `end=" "` argument to `print` keeps the cursor from going to the next line when we don't want it to do so. We'll use an empty `print` to get to the next line when we do.

It'll be hard for a player to determine the position of the pieces by index, so lets adopt a scheme where the rows are specified by a letter and the row by a number. Let's also change our `draw_board` to put this information on the board. 

In [ ]:
row_names = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
row_map = dict(zip(row_names,range(size)))

row_map

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7}

In [ ]:
column_names = list(map(str,range(1,size+1)))
column_map = dict(zip(column_names,range(size)))

column_map

{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7}

In [ ]:
def draw_board(board):
    print(" ",end=" ")
    for i in range(size):
        print(column_names[i],end=" ")
    print()
    
    for i in range(size):
        print(row_names[i],end=" ")
        for j in range(size):
            print(space_character[board[i][j]],end=" ")
        print()      
    

But now the user will give us locations like "C3" and we'll have to convert it to a pair of indexes for the matrix. These are stored in the `row_map` and `column_map` dictionaries, but we can't trust the user to correctly supply an input, so lets be careful by checking the type of input, length, and making sure it's upper case:

In [ ]:
def parse_location(location_string):
    
    if not isinstance(location_string,str):
        print_message("Invalid input. Location must be a string.")
        return False
    
    if len(location_string)!=2:
        print_message("Inavlid input. Location must be 2 characters")
        return False
    
    row = location_string[0].upper()
    col = location_string[1].upper()
    
    if not row in row_map:
        print_message("Not a valid row.")
        return False
    
    if not col in column_map:
        print_message("Not a valid column.")
        return False
    
    return row_map[row], column_map[col]

Similarily setup things so the user can specify "L" or "R" for the moves:

In [ ]:
def parse_move(move_string):
    if not isinstance(move_string,str):
        print_message("Invalid input. Location must be a string.")
        return False
    
    if len(move_string)!=1:
        print_message("Inavlid input. Location must be 2 characters")
        return False
    
    if move_string.upper() == "L":
        return left_move
    
    if move_string.upper() == "R":
        return right_move
    
    print_message("Bad Move. Must input R or L.")
    return -1

In [ ]:
def nice_move_piece(board,player,location_string,move_string):
    location = parse_location(location_string)
    move = parse_move(move_string)

    if location and move!=-1:
        return move_piece(board,player,location,move)
    else:
        print_message("Invalid move.")
        return False

## Making a full game
Next we don't want the player to be making python calls, the game should show each player the board and ask for input. They should just enter a position and a direction. And the game should keep going until someone wins.

We'll use python's `input` to interact with the player:

In [ ]:
def take_move(board,player):
    good_move = False
    
    while not good_move:
        location_string = input("Input Location (to stop the game, input XX:")
        if location_string=="XX":
            good_move = "end"
            return good_move
        
        move_string = input("Input Move (L/R):")
        
        good_move = nice_move_piece(board,player,location_string,move_string)

Now lets write a function that determines if the game is won, so it can stop:

In [ ]:
## Make a function to check the win state of the board

#Make a function to count number of each player's pieces
def count_piece(board,player):
    n = 0
    for i in range(size):
        for j in range(size):
            if board[i][j]==player:
                n+=1
    return n

def game_won(board):
    player_1_n = count_piece(board,player_1)
    player_2_n = count_piece(board,player_2)
    
    if player_1_n==0:
        return player_2
    if player_2_n==0:
        return player_1
    
    return False    

Pulling all of these functions together, we get a basic checkers game. Note that it doesn't handle king pieces. And it won't realize when the game is a stale mate. 

In [ ]:
def checkers_game():
    print("Welcome to Checkers!")
    print("--------------------")
    
    board_0 = make_game_board()
    
    player = player_1
    
    this_game_won = False
    
    while not this_game_won:
        draw_board(board_0)
        
        print("Player",player,"move:")
        new_move = take_move(board_0,player)
        
        if new_move == "end":
            print("Player has exited the game")
            break
        
        this_game_won = game_won(board_0)
        
        if player==player_1:
            player = player_2
        else:
            player = player_1
        
    if not new_move == "end":
        print("Winner is player",this_game_won)

In [ ]:
checkers_game()

Welcome to Checkers!
--------------------
Input board size:9
Error: board size must be an even integer greater than or equal to 8
Input board size:8
Error: board size must be an even integer greater than or equal to 8
Input board size:8
Error: board size must be an even integer greater than or equal to 8
Input board size:8
Error: board size must be an even integer greater than or equal to 8
